In [1]:
import requests
import json

def make_request(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

In [2]:
import json


# Fetch spot prices
spot_prices = {}

for month in range(1, 13): # 1-13
    month_str = str(month).zfill(2)
    spot_prices[month_str] = {}
    for day in range(1, 32):
        day_str = str(day).zfill(2)
        url = f"https://www.hvakosterstrommen.no/api/v1/prices/2024/{month_str}-{day_str}_NO1.json"
        data = make_request(url)
        if data:
            spot_prices[month_str][day_str] = [hour["NOK_per_kWh"] for hour in data]



In [ ]:
month = "11"  # January
day = "15"    # 15th day
if month in spot_prices and day in spot_prices[month]:
    daily_prices = spot_prices[month][day]
    # Process daily_prices as needed
    print(daily_prices)


print(json.dumps(spot_prices['01']['01'], indent=4))

### Her må det legges inn data

In [48]:
# Get price data for month
monthly_spot_fortum = {
    "01": 0.9716,
    "02": 0.7412,
    "03": 0.7477,
    "04": 0.6556,
    "05": 0.4238,
    "06": 0.4639,
    "07": 0.6433,
    "08": 0.5922,
    "09": 0.6961,
    "10": 0.9536,
    "11": 1.0881,
    "12": 1.1844
}


# period = '02' 
# Make a list of the days in the month. Make sure to have the right number of days (Days + 1).
# days = [str(day).zfill(2) for day in range(1, 32)]


### Denne må endres til riktig fil

In [3]:
import json
# Get the file for the month
with open(f'Forbruk_2023-12-31_til_2024-12-31.json', 'r') as file:
    strom_data = json.load(file)

# Now `strom_data` is a Python dictionary that contains the parsed JSON data

In [4]:
# Assuming strom_data is already loaded and spot_prices is already populated

# Calculate the Cost for each hour as shown previously
for month in strom_data["Years"][0]["Months"]:
    month_str = str(month["Month"]).zfill(2)
    for day in month["Days"]:
        day_str = str(day["Day"]).zfill(2)
        if month_str in spot_prices and day_str in spot_prices[month_str]:
            spot_prices_for_day = spot_prices[month_str][day_str]
            last_price = 0 # Initialize last price to 0
            for hour_index, hour in enumerate(day["Hours"]):
                if hour_index < len(spot_prices_for_day):
                    spot_price = spot_prices_for_day[hour_index]
                    last_price = spot_price # Set to use when there is no price for the hour
                    consumption = hour["Consumption"]["Value"]
                    hour["SpotPrice"] = spot_price
                    hour["Cost"] = spot_price * consumption
                else:
                    consumption = hour["Consumption"]["Value"]
                    hour["SpotPrice"] = last_price
                    hour["Cost"] = last_price * consumption
                    print(hour)
                    print(len(spot_prices_for_day))
                    print(f"Skipping hour {hour_index} on {day_str}-{month_str} due to daylight saving time adjustment")

{'Year': 2024, 'Month': 3, 'Day': 31, 'Hour': 23, 'IsPublicHoliday': False, 'UseWeekendPrices': False, 'EnergyPrice': None, 'FixedPriceLevel': None, 'TotalPriceCost': None, 'Distribution': None, 'HourCompensation': None, 'SpotPrice': 0.64616, 'EnergyPriceCost': None, 'FixedPriceCost': None, 'PowerSupplierCost': None, 'IsPeriodCompleted': False, 'Id': '2024033123', 'Production': None, 'Consumption': {'Value': 3.13, 'IsVerified': True, 'Status': 'OK'}, 'Cost': 2.0224808}
23
Skipping hour 23 on 31-03 due to daylight saving time adjustment


In [ ]:
timer = strom_data["Years"][0]["Months"][2]["Days"][0]["Hours"]
dager = strom_data["Years"][0]["Months"][2]["Days"]
print(len(timer))
print(len(dager))
print(strom_data["Years"][0]["Months"][2]["Days"][0]["Hours"][0]["Consumption"]["Value"])
print(json.dumps(strom_data["Years"][0]["Months"][2]["Days"][30]["Hours"][0], indent=2))

In [7]:
# Sum the total cost for January
total_cost_march = 0

# January is the first month, so we use index 0
march = strom_data["Years"][0]["Months"][2]

for day in march["Days"]:
    # print(day['Day'])
    for hour in day["Hours"]:
        total_cost_march += hour["Cost"]

print(f"Total Cost for march: {total_cost_march:.2f} NOK")
print(len(march))

Total Cost for march: 1248.01 NOK
21


In [8]:
march = strom_data["Years"][0]["Months"][2]
print(json.dumps(strom_data["Years"][0]["Months"][2]["Days"][30]['TotalPriceCost'], indent=2))

{
  "Total": 0.0,
  "TotalExVat": 0.0,
  "TotalExTaxes": 0.0,
  "MonetaryUnitOfMeasure": null,
  "Currency": "NOK"
}


In [ ]:
# Sum the hourly costs and set the TotalPriceCost for each day
for month in strom_data["Years"][0]["Months"]:
    for day in month["Days"]:
        print(f"Processing day: {day['Year']}-{day['Month']:02d}-{day['Day']:02d}")
        total_daily_cost = sum(hour["Cost"] for hour in day["Hours"])
        day["TotalPriceCost"]["TotalExVat"] = total_daily_cost
        day["TotalPriceCost"]["Total"] = total_daily_cost * 1.25

# Sum the daily costs and set the MonthlyTotalCost for each month
for month in strom_data["Years"][0]["Months"]:
    total_monthly_cost = sum(day["TotalPriceCost"]["TotalExVat"] for day in month["Days"])
    month["TotalPriceCost"]["TotalExVat"] = total_monthly_cost
    month["TotalPriceCost"]["Total"] = total_monthly_cost * 1.25

# Sum the Monthly costs and set the YearlyTotalCost for each month
# Sum the monthly costs to get the yearly total cost
total_yearly_cost = sum(month["TotalPriceCost"]["TotalExVat"] for month in strom_data["Years"][0]["Months"])
year = strom_data["Years"][0]
year["TotalPriceCost"]["TotalExVat"] = total_yearly_cost
year["TotalPriceCost"]["Total"] = total_yearly_cost * 1.25



In [26]:
print(json.dumps(strom_data["Years"][0]["Months"][3]["Days"][0]['TotalPriceCost'], indent=2))

print(json.dumps(strom_data["Years"][0]["Months"][10]['TotalPriceCost'], indent=2))

print(json.dumps(strom_data["Years"][0]['TotalPriceCost'], indent=2))

{
  "Total": 45.9328965,
  "TotalExVat": 36.7463172,
  "TotalExTaxes": 0.0,
  "MonetaryUnitOfMeasure": null,
  "Currency": "NOK"
}
{
  "Total": 1292.5290042499998,
  "TotalExVat": 1034.0232033999998,
  "TotalExTaxes": 0.0,
  "MonetaryUnitOfMeasure": null,
  "Currency": "NOK"
}
{
  "Total": 12829.679324375,
  "TotalExVat": 10263.7434595,
  "TotalExTaxes": 0.0,
  "MonetaryUnitOfMeasure": null,
  "Currency": "NOK"
}


In [28]:
print(json.dumps(strom_data["Years"][0]["Months"][10]['TotalPriceCost']["TotalExVat"]))
print(json.dumps(strom_data["Years"][0]["Months"][10]['Consumption']["Value"]))

1034.0232033999998
1903.3


In [ ]:
import calendar

for month in strom_data["Years"][0]["Months"]:
  month_number = month["Month"]
  month_name = calendar.month_name[month_number]
  month["MonthName"] = month_name

# Print the updated months to verify
for month in strom_data["Years"][0]["Months"]:
  print(f"Month Number: {month['Month']}, Month Name: {month['MonthName']}")

In [38]:
for month in strom_data["Years"][0]["Months"]:
    
    total_price = month["TotalPriceCost"]["TotalExVat"]
    total_consumption = month["Consumption"]["Value"]
    month['SpotPrice'] = total_price / total_consumption

In [ ]:
# Print the price and consumption for each month
period = 1
overprice = 0
for month in strom_data["Years"][0]["Months"]:
    month_name = month["MonthName"]
    total_price = month["TotalPriceCost"]["TotalExVat"]
    total_consumption = month["Consumption"]["Value"]
    fortum_price = monthly_spot_fortum[str(period).zfill(2)] * total_consumption
    period += 1
    spot_price = month["SpotPrice"]
    delta_price = fortum_price - total_price
    overprice += delta_price
    print(f"Month: {month_name}")
    print(f" Total Consumption: {total_consumption:.2f} kWh")
    print(f" My Spot Price: {spot_price:.4f} NOK/kWh")
    print(f" Total Cost (Excluding VAT): {total_price:.2f} NOK")
    print(f" Fortum Cost (Excluding VAT): {fortum_price:.2f} NOK")
    print(f" Fortum Cost - My Total Cost: {delta_price:.2f} NOK")
    
    print("-" * 40)

print(f"Year: {strom_data['Years'][0]['Year']}")
print(f'  Total Price (Excluding VAT): {year["TotalPriceCost"]["TotalExVat"]:.2f} NOK')
print(f'  Total Consumption: {year["Consumption"]["Value"]:.2f} kWh')

print(f'  Total OverPrice (Excluding VAT) for 2024: {overprice:.2f} NOK')
print("-" * 40)



In [53]:
with open("output.txt", "w") as f:
  # Print the price and consumption for each month
  period = 1
  overprice = 0
  for month in strom_data["Years"][0]["Months"]:
      month_name = month["MonthName"]
      total_price = month["TotalPriceCost"]["TotalExVat"]
      total_consumption = month["Consumption"]["Value"]
      fortum_price = monthly_spot_fortum[str(period).zfill(2)] * total_consumption
      period += 1
      spot_price = month["SpotPrice"]
      delta_price = fortum_price - total_price
      overprice += delta_price
      f.write(f"Month: {month_name}")
      f.write(f" Total Consumption: {total_consumption:.2f} kWh")
      f.write(f" My Spot Price: {spot_price:.4f} NOK/kWh")
      f.write(f" Total Cost (Excluding VAT): {total_price:.2f} NOK")
      f.write(f" Fortum Cost (Excluding VAT): {fortum_price:.2f} NOK")
      f.write(f" Fortum Cost - My Total Cost: {delta_price:.2f} NOK")
      
      f.write("-" * 40)

  f.write(f"Year: {strom_data['Years'][0]['Year']}")
  f.write(f'  Total Price (Excluding VAT): {year["TotalPriceCost"]["TotalExVat"]:.2f} NOK')
  f.write(f'  Total Consumption: {year["Consumption"]["Value"]:.2f} kWh')

  f.write(f'  Total OverPrice (Excluding VAT) for 2024: {overprice:.2f} NOK')
  f.write("-" * 40)
  

In [45]:
with open('strom_data_output.json', 'w') as outfile:
  json.dump(strom_data, outfile, indent=4)

In [ ]:
print(type(strom_data))
print(type(strom_data["Years"][0]["Months"][0]["Days"]))

In [ ]:
# Check if all days have 24 hours
print(month_prices["01"][0]["NOK_per_kWh"])
len(month_prices["25"])
for day in month_prices:
    print(len(month_prices[day]))

In [ ]:
# Set the strom_data variable to the parsed JSON data
strom_data_list:list = []
strom_data_list = strom_data["Years"][0]["Months"][1]["Days"]
print(len(strom_data_list))
print(strom_data_list[0])

In [ ]:
# Totalt antall kilowattimer i month
total_days = 0
total_hours = 0
total = 0

for day in strom_data_list:
    total_days += 1
    # add a new key to the dictionary
    for hour in day["Hours"]:
        total_hours += 1
        total += hour["Consumption"]["Value"]

print(total_days)
print(total_hours)
print(f'Totalt antall kwh: {total:.2f}')


In [ ]:
print(strom_data_list[0]["Hours"])
print (len(strom_data_list[0]["Hours"]))

In [ ]:
price_object = {}
for day in month_prices:
  hour_prices = {}
  for i, hour in enumerate(month_prices[day]):
    hour_prices[i] = hour["NOK_per_kWh"]
    # print(i, hour["NOK_per_kWh"])
  price_object[day] = hour_prices

print(len(price_object))



In [90]:
for day in strom_data_list:

    day_num = str(day["Day"]).zfill(2)
    for i, hour in enumerate(day["Hours"]):
        hour["SpotPrice"] = price_object[day_num][i]

In [ ]:
print(strom_data[0]["Hours"][10])

In [ ]:
day_data = []
for i, day in enumerate(strom_data_list):
    kwh = 0
    cost = 0
    for hour in day["Hours"]:
        kwh += hour["Consumption"]["Value"]
        cost += hour["Consumption"]["Value"] * hour["SpotPrice"]
    day_data.append({"Day": day["Day"], "KWH": kwh, "Cost": cost})
print(day_data[:1])
    

In [ ]:
# My actual consumption

# Calculate total cost and total kwh for the month
total_kwh = 0
total_cost = 0
num_days = 0
for day in day_data:
    num_days += 1
    total_kwh += day["KWH"]
    total_cost += day["Cost"]

# Calculate average cost per kwh
average_cost = total_cost / total_kwh

# Calculate average price per kwh
average_price = total_cost / num_days

total_cost_incl_mva = total_cost * 1.25
average_cost_incl_mva = average_cost * 1.25

print(f"Totalt antall dager: {num_days}")
print(f"Totalt antall kwh: {total_kwh:.2f}")
print(f"Totalt antall kroner: {total_cost:.2f}")
print(f"Totalt antall kroner inkl mva: {total_cost_incl_mva:.2f}")
print(f"Gjennomsnittlig pris per kwh forbrukt i øre: {average_cost*100:.2f}")
print(f"Gjennomsnittlig pris per kwh ink mva i øre: {average_cost*1.25*100:.2f}")
print(f"Gjennomsnittlig pris for måneden per dag: {average_price:.2f}")



In [ ]:
import matplotlib.pyplot as plt

# Extract the day and consumption values from the day_data list
days = [day["Day"] for day in day_data]
consumption = [day["KWH"] for day in day_data]
cost = [day["Cost"] for day in day_data]

# Plot the bar chart for consumption
plt.bar(days, consumption, label='Consumption')

# Plot the line chart for cost
plt.plot(days, cost, color='r', label='Cost')

plt.xlabel("Day")
plt.ylabel("Consumption / Cost")
plt.title("Daily Consumption and Cost")
plt.legend()
plt.show()


In [ ]:
import sys
print(sys.executable)

In [ ]:
print(month_prices["01"][1]["NOK_per_kWh"])
price_01 = 0
hours_01 = 0
# Get the average price for the given day of month including VAT
for price in month_prices["19"]:
    hours_01 += 1
    price_01 += price["NOK_per_kWh"]
print(f'Gjennomsnitt per dag ex mva: {(price_01 / hours_01)}')  
print(f'Gjennomsnitt per dag ink mva: {(price_01 / hours_01)*1.25}')    

In [ ]:
# Calculate the average price and cost for the month using the average spotprice 

day_average = []
hours_month = 0
cost = 0
for i, day in enumerate(strom_data_list):
    for hour in day["Hours"]:
        hours_month += 1
        cost += hour["SpotPrice"]
    day_average.append({"Day": day["Day"], "KWH": kwh, "Cost": cost})

daily_average = cost / hours_month*100
daily_average_incl_mva = daily_average * 1.25

print(day_average[0])
print(hours_month)
print("Ved gjennomsnittlig spotpris:")
print(f"Gjennomsnittlig pris per kwh: {daily_average:.2f}")
print(f"Gjennomsnittlig pris per kwh ink mva i øre: {daily_average_incl_mva:.4f}")
print(f'Totale kostnader ved gjennomsnittspris: {daily_average*total_kwh/100:.2f}')
print(f'Totale kostnader ved gjennomsnittspris ink mva: {daily_average_incl_mva*total_kwh/100:.2f}')
# print(cost/hours_month)



In [ ]:
print(f"Nummer for måneden: {period}")
print(f"Totalt antall dager i måneden: {num_days}")
print(f"Totalt antall kwh: {total_kwh:.2f}\n")

print("\033[1m" + "Mitt faktiske forbruk:" + "\033[0m")
print(f"Totalt antall kroner: {total_cost:.2f}")
print(f"Totalt antall kroner inkl mva: {total_cost_incl_mva:.2f}\n")

print(f"Gjennomsnittlig pris per kwh forbrukt i øre: {average_cost*100:.2f}")
print(f"Gjennomsnittlig pris per kwh ink mva i øre: {average_cost*1.25*100:.2f}")
print(f"Gjennomsnittlig pris for måneden per dag: {average_price:.2f}\n")

print("\033[1m" + "Ved gjennomsnittlig spotpris:" + "\033[0m")
print(f'Totale kostnader ved gjennomsnittspris: {daily_average*total_kwh/100:.2f}')
print(f'Totale kostnader ved gjennomsnittspris ink mva: {daily_average_incl_mva*total_kwh/100:.2f}\n')
print(f"Gjennomsnittlig pris per kwh: {daily_average:.2f}")
print(f"Gjennomsnittlig pris per kwh ink mva i øre: {daily_average_incl_mva:.4f}\n")

print("\033[1m" + "Delta mellom fakturert og faktisk ved spot:" + "\033[0m")
print(f"Totalt antall kroner uten mva: {(total_cost-price_from_fortum_per_kwh*total_kwh):.2f}")
print(f"Totalt antall kroner inkl mva: {(total_cost_incl_mva-price_from_fortum_per_kwh*total_kwh*1.25):.2f}\n")


In [ ]:
print("\033[1m" + "Mitt faktiske forbruk:" + "\033[0m")